# Demo notebook to start working on the GBM Hackathon data
This notebook demonstrates how the MOSAIC ([Owkin](https://www.mosaic-research.com/)) and BRUCE ([PICI](https://www.parkerici.org/)) data can be loaded.

## Setup

In [ ]:
# Load packages and classes
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tiffslide
import seaborn as sns
import gget
import tifffile
import zarr

# MosaicDataset and BruceDataset classes allow loading and visualisation of the different data sources
from gbmhackathon import MosaicDataset, BruceDataset

## Start exploring the MOSAIC data

In [ ]:
# Look at the available data sources
MosaicDataset.sources.keys()

In [ ]:
# The following cell will returns a dictionary with the data sources in the
# keys and the list of files (and path) used to return the data.
source_dict_mosaic = MosaicDataset.load_tabular()

### Load the MOSAIC sample table
Not every data modality is available for each MOSAIC sample. The following table provides information on which modality is available for each sample.

In [ ]:
filename_sample_table = "/home/ec2-user/SageMaker/data/mosaic_dataset/Data availibility per modality per patient.csv"
sample_table = pd.read_csv(filename_sample_table, index_col=0)
sample_table.head(2)

### Load and explore the six MOSAIC data modalities

#### 1. Clinical data

In [ ]:
# To access the clinical data, specify the correct keys
clin_df = source_dict_mosaic["clinical"]["processed gbm clinical"]
clin_df.head(2)

In [ ]:
# For the clinical data, you also have access to a dictionary to understand the
# variable's name
data_dict = source_dict_mosaic["clinical"]["data dictionary"]
data_dict.head(3)

#### 2. Bulk RNA sequencing data

In [ ]:
# To access the bulk RNAseq data, you can specify a particular normalization
print(source_dict_mosaic["bulk_rna"].keys())
bRNA_TPM = source_dict_mosaic["bulk_rna"]["TPM counts"]
bRNA_TPM.head(2)

In [ ]:
# A quick way to convert ensemblID to gene names, using gget, but other tools exist
# ensembl_ids = bRNA_TPM.index.astype(str).tolist()[:10]
# result = gget.info(ensembl_ids, verbose=False)
# result["ensembl_gene_name"]

#### 3. Whole exome sequencing (WES) data

The single nucleotide variant (SNV) and small insertion and deletion (indel) information are stored in a DataFrame, with rows representing samples and columns representing genes. Each cell contains a Boolean value (True/False) indicating whether a gene contains a potentially oncogenic SNV or indel in that sample.

A gene is considered to have a potentially oncogenic alteration if it meets one of these criteria:
- The variant creates a nonsense mutation in a known tumour suppressor gene (TSG)
- The variant results in a previously documented amino acid change at a known cancer hotspot

Note that the TSGs and hotspots used in this analysis are not specific to GBM. To focus specifically on likely GBM drivers, you may want to use a more restricted gene list.

WES was performed on tumour samples only and we are therefore not always able to distinguish between somatic and germline variants. Some genes, such as HLA genes, likely contain false positives.

In [ ]:
# Access SNVs and indels
snvs_indels = source_dict_mosaic["wes"]["WES mutations"]
snvs_indels.head(2)

In [ ]:
snvs_indels.sum().sort_values(ascending=False).head()

Copy number variant (CNV) information is stored across three distinct DataFrames:
- A binary matrix indicating which genes are affected by deletions in each sample
- A binary matrix indicating which genes are affected by amplifications in each sample
- A binary matrix indicating which genes are affected by potentially oncogenic CNVs in each sample

A gene is considered to be affected by a potentially oncogenic CNV if it meets either of these criteria:
- It is a known TSG and is affected by a deletion
- It is a known oncogene and is affected by a duplication

The lists of known TSGs and oncogenes are obtained from the IntOGen database. While TSGs and oncogenes can be cancer-specific, our analysis considers all TSGs and oncogenes identified in any cancer type, not just those specific to GBM.

In [ ]:
# Access amplifications
CNVamp = source_dict_mosaic["wes"]["WES CNV amplification"]
CNVamp.head(2)

In [ ]:
# You can also access the deletions
CNVdel = source_dict_mosaic["wes"]["WES CNV deletion"]
CNVdel.head(2)

In [ ]:
print("Number of samples with information on CNV alterations: ", CNVdel.shape[0])

In [ ]:
# Show the top deleted genes
CNVdel.sum().sort_values(ascending=False).head()

In [ ]:
# Access the potentially oncogenic CNVs and show the top genes
CNVoncogenic = source_dict_mosaic["wes"]["WES CNV oncogenic"]
CNVoncogenic.sum().sort_values(ascending=False).head()

#### 4. Single-cell RNA sequencing (Chromium) data
Processed single-cell RNA sequencing data are stored as an AnnData object. Ambient RNA has been removed using [SoupX](https://academic.oup.com/gigascience/article/9/12/giaa151/6049831) and doublets identified and removed using [ScDblFinder](https://pmc.ncbi.nlm.nih.gov/articles/PMC9204188/).

In [ ]:
# Note that it can take up to 12 minutes to load the single-cell data because it is heavy
single_cell_obj = MosaicDataset.load_singlecell()
# Display the content of the anndata object
single_cell_obj.__dict__.keys()

Various normalisations have been applied to the data. The unnormalised and normalised gene expression data can be accessed as follows.

In [ ]:
# .layers["ambient_rna_removed"] contains the unnormalised counts
single_cell_obj.layers["ambient_rna_removed"]

In [ ]:
# .layers["LogNormalize"] contains log-normalised counts
single_cell_obj.layers["LogNormalize"]

In [ ]:
# .X contains the log of the SCTransform-corrected counts plus one
single_cell_obj.X

The `.obs` property contains cell-level meta-data. Each row represents a cell. The most useful columns are:
- `orig.ident` The sample ID.
- `celltype_level1_scanvi` Level 1 cell type annotations. These are the annotations at the lowest resolution, and comprise the following five cell types: *Immune*, *Malignant*, *Neuroglia*, *Neuron*, *Stromal*.
- `celltype_level2_scanvi` Level 2 cell type annotations. These are higher resolution than level 1 annotations.
- `celltype_level3_scanvi` Level 3 cell type annotations. Malignant cells from each sample are annotated separately, for example, *Tu_HK_G_001* are malignant cells from sample *HK_G_001*.
- `celltype_level4_scanvi` Level 4 cell type annotations. These are the highest-resolution annotations. Malignant cells from each sample have been subclustered based on transcriptional similarity. For example, malignant cells from *Tu_HK_G_111b* have been subclustered into *Tu_HK_G_111b_c01*, *Tu_HK_G_111b_c02*, *Tu_HK_G_111b_c03*, and *Tu_HK_G_111b_nos*.

In [ ]:
single_cell_obj.obs.head(2)

PCA and UMAP embedding have already been computed. These are stored in the `.obsm` property.

In [ ]:
# Visualize single-cell UMAP coloured by level 2 cell type annotations
embedding = single_cell_obj.obsm["X_umap"][:, :2]
df = pd.DataFrame(embedding, columns=["UMAP1", "UMAP2"])
df["cell_type"] = single_cell_obj.obs["celltype_level2_scanvi"].values
sns.scatterplot(
    data=df,
    x="UMAP1",
    y="UMAP2",
    hue="cell_type",
    palette="tab10",
    s=10,
    alpha=0.7
)
plt.title("UMAP Embedding with level 2 cell type annotations")
plt.show()

#### 5. Spatial transcriptomic (Visium) data
The `notebooks/visium_starter_mosaic.ipynb` Notebook provides more information on how to process and explore Visium data.

In [ ]:
# Load Visium data
visium_obj = MosaicDataset.load_visium(
    sample_list=["HK_G_022a_vis", "HK_G_024a_vis", "HK_G_030a_vis"], # remove this argument to load all available samples
    resolution="hires"
)

#### 6. Aligned H&E images

In [ ]:
# Load the H&E slides
he = source_dict_mosaic["he"]["HE files"]
he.head(2)

In [ ]:
# Display a H&E slide
slide_idx = 12  # pick a H&E slide to display

slide = tiffslide.TiffSlide(he.path.values[slide_idx])

# Display the slide
slide_img = slide.get_thumbnail(slide.level_dimensions[-2])
fig, ax = plt.subplots()
ax.imshow(slide_img)
ax.set_axis_off()
plt.show()

In [ ]:
# Load the H1 Bioptimus features paths for each subject
h1 = source_dict_mosaic["he"]["H1 features"]
h1.head(2)

In [ ]:
# Load the H1 features for one sample

slide_idx = 12  # pick a H&E slide to display

# Load the H1 zarr object for the selected slide
h1_zarr = zarr.open(h1.path.values[slide_idx], mode='r')

h1_emb = h1_zarr["emb"][:]  # (n tiles, n features) np array H1 features
h1_coords = h1_zarr["coords"][:]  # (n tiles, 2) np array coordinates
h1_level = h1_zarr["level"][:]  # (n tiles, 1) np array level of resolution


## Start exploring the BRUCE Data

In [ ]:
# Look at the available data sources
BruceDataset.sources.keys()

In [ ]:
# Load the metadata table
source_dict_bruce = BruceDataset.load_tabular()

In [ ]:
metadata = source_dict_bruce["metadata"]["metadata"]
metadata.head(2)

### Load a MIBI image

In [ ]:
source_dict_bruce.keys()

In [ ]:
# Load the information on the image
mibi_immune = source_dict_bruce["mibi_images"]["immune"] # you can also choose "tumor"
mibi_immune.head(2)

In [ ]:
# Display for one sample and one immune marker of interest
img_idx = 1
img = tifffile.imread(mibi_immune.path.values[img_idx])

# Normalize the image to the range [0, 1]
img_normalized = (img - img.min()) / (img.max() - img.min())

# Display the normalized image
plt.imshow(img_normalized)
plt.axis('off')
plt.show()